In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['HOME_DIR'] = 'drive/MyDrive/hidden-networks'
# !pip install -r $HOME_DIR/requirements.txt

import sys
sys.path.append(os.path.join('/content', os.environ['HOME_DIR']))

!pip install optuna
import optuna

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.autograd as autograd
import collections

from supermask_pruning import GetSubnet, SupermaskConv, SupermaskLinear
from supermask_pruning import train, test

class ArgClass:
    def __init__(self, args):
        self.setattrs(**args)
        
    def setattrs(self, **kwargs):
        for name, val in kwargs.items():
            setattr(self, name, val)

In [4]:
class Net(nn.Module):
    def __init__(self, args, input_channels, image_size, num_labels):
        super().__init__()
        
        sparsities = getattr(args, "sparsity", [{"sparsity": 1.0}, {"sparsity": 1.0}, {"sparsity": 1.0}, {"sparsity": 1.0}, {"sparsity": 1.0}])
        self.conv1 = SupermaskConv(input_channels, 64, 3, 1, bias=args.bias, init=args.init, **sparsities[0])
        self.conv2 = SupermaskConv(64, 64, 3, 1, bias=args.bias, init=args.init, **sparsities[1])
        s = (image_size - 4) * (image_size - 4) * 64 // 4
        self.fc1 = SupermaskLinear(s, 256, bias=args.bias, init=args.init, **sparsities[2])
        self.fc2 = SupermaskLinear(256, 256, bias=args.bias, init=args.init, **sparsities[3])
        self.fc3 = SupermaskLinear(256, num_labels, bias=args.bias, init=args.init, **sparsities[4])
        self.args = args

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def get_extra_state(self):
        return self.args
      
    def set_extra_state(self, state):
        self.args = state

In [ ]:
args = {"init": "signed_constant", "bias": False}
for i in Net(ArgClass(args), 3, 32, 10).children():
  print(i.weight)

In [12]:
global failed_model
failed_model = None

In [13]:
# The main function runs the full training loop on a dataset of your choice
def main(model_args, train_args, base_model=None, trial=None):
    args = ArgClass(model_args)
    train_args = ArgClass(train_args)
    dataset = args.dataset

    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")
    print(f"Using device {device}")

    transform = None
    if dataset == "MNIST":
        transform = transforms.Compose([transforms.ToTensor(), 
                                        transforms.Normalize((0.1307,), (0.3081,))
                                        ])
        train_transform = transform
        input_channels, image_size, num_labels = 1, 28, 10
    elif dataset == "CIFAR10":
        train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                              transforms.RandomHorizontalFlip(),
                                              transforms.ToTensor(),
                                              transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                              ])
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                        ])
        input_channels, image_size, num_labels = 3, 32, 10
    else:
        raise ValueError("Only supported datasets are CIFAR10 and MNIST currently.")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        getattr(datasets, dataset)(os.path.join(train_args.data, dataset), 
                                   train=True, download=True, transform=transform),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    train_augmented_loader = torch.utils.data.DataLoader(
        getattr(datasets, dataset)(os.path.join(train_args.data, dataset), 
                                   train=True, transform=train_transform),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        getattr(datasets, dataset)(os.path.join(train_args.data, dataset), 
                                   train=False, transform=transform),
        batch_size=train_args.test_batch_size, shuffle=True, **kwargs)

    model = Net(args, input_channels, image_size, num_labels).to(device)

    if getattr(args, "copy_layers", None) is not None:
        if (bool(args.copy_layers) ^ (base_model is not None)):
            raise ValueError("copy_layers arg must be None or [] if base_model is not specified")
        if base_model is not None and args.copy_layers:
            for layer in args.copy_layers:
                model.load_state_dict(getattr(base_model, layer).state_dict(prefix=f"{layer}."), strict=False)
            
    # NOTE: only pass the parameters where p.requires_grad == True to the optimizer! Important!
    optimizer = getattr(optim, args.optimizer)(
        [p for p in model.parameters() if p.requires_grad],
        **args.optim_kwargs,
    )
    assert isinstance(args.epochs, list) or isinstance(args.epochs, int)
    num_epochs, check_freeze = (args.epochs, False) if isinstance(args.epochs, int) else (max(args.epochs), True)
    criterion = nn.CrossEntropyLoss().to(device)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs) if args.scheduler else None 

    for epoch in range(1, num_epochs + 1):
        if check_freeze:
            for freeze_at_epoch, child in zip(args.epochs, model.children()):
                if freeze_at_epoch == epoch - 1:
                    child.freeze()
                    print(f"Freezing {child} before epoch {epoch}")
        try:
            train(model, train_args.log_interval, device, train_augmented_loader, optimizer, criterion, epoch)
        except Exception as e:
            global failed_model
            failed_model = model
            raise e
        if (train_args.train_eval_interval and epoch % train_args.train_eval_interval == 0) or (train_args.eval_on_last and epoch == args.epochs):
            train_acc, train_loss = test(model, device, criterion, train_loader, name="Train")
            if trial:
                trial.set_user_attr('train_acc', {**trial.user_attrs.get('train_acc', {}), **{epoch: train_acc}})
                trial.set_user_attr('train_loss', {**trial.user_attrs.get('train_loss', {}), **{epoch: train_loss}})
        if (train_args.test_eval_interval and epoch % train_args.test_eval_interval == 0) or (train_args.eval_on_last and epoch == args.epochs):
            test_acc, test_loss = test(model, device, criterion, test_loader, name="Test")
            if trial:
                trial.set_user_attr('test_acc', {**trial.user_attrs.get('test_acc', {}), **{epoch: test_acc}})
                trial.set_user_attr('test_loss', {**trial.user_attrs.get('test_loss', {}), **{epoch: test_loss}})
                trial.report(test_acc, epoch-1)
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

        if scheduler:
            scheduler.step()

    if args.save_name is not None:
        torch.save(model.state_dict(), os.path.join(os.environ['HOME_DIR'], \
                                                    "trained_networks", args.save_name))
    
    return model, device, train_loader, test_loader, criterion

def get_prune_mask(layer, sparsity):
    with torch.no_grad():
        return GetSubnet.apply(layer.scores.abs(), sparsity)

In [ ]:
# # Arguments that do not affect model at all
train_args = {
    "test_batch_size": 1000, # input batch size for testing (default: 1000)
    'data': '../data', # Location to store data (e.g. MNIST)
    'log_interval': 500000, # how many batches to wait before logging training status
    'train_eval_interval': 10, # epoch interval at which to print training accuracy
    'test_eval_interval': 10, # epoch interval at which to print test accuracy
    'eval_on_last': True
}

args = {
  "dataset": "CIFAR10",
  "init": "signed_constant",
  "batch_size": 128, # input batch size for training (default: 64)
  "epochs": 100, # number of epochs to train (default: 14)
  "optimizer": "SGD",
  "optim_kwargs": {"lr": 0.1, "momentum": 0.9, "weight_decay": 0.0001},
  "scheduler": True, # False for Adam, True for SGD, does CosineAnnealing
  'no_cuda': False, # disables CUDA training
  'seed': 1000, # random seed (default: 1)
  'save_name': None, #"conv2_frozen_sp50_rs1000", # "simple20_rs2", # For Saving the current Model, None if not saving
  'sparsity': [{"threshold": 0.2, "min_sparsity": 0.05} for i in range(5)], # 'how sparse is each layer'
  'copy_layers': [], # ['conv1', 'conv2', 'fc2'],
  'bias': False
}

# trained_model, device, train_loader, test_loader, criterion = main(args, train_args)
name_of_experiment = 'threshold-100epoch'
train_results = []
test_results = []
args['seed'] = 282
for threshold in [0.02, 0.05, 0.1, 0.2, 0.5, 0.01]:
    args['sparsity'] = [{"threshold": threshold, "min_sparsity": 0.05} for i in range(5)]
    args['save_name'] = f'FINAL_thresholds_{threshold}'
    print(f"----{args['seed']}----{args}----")
    trained_model, device, train_loader, test_loader, criterion = main(args, train_args)
    train_acc, train_loss = test(trained_model, device, criterion, train_loader, name="Train")
    test_acc, test_loss = test(trained_model, device, criterion, test_loader)
    train_results.append((threshold, train_acc, train_loss))
    test_results.append((threshold, test_acc, test_loss))
    torch.save((train_args, args, train_results, test_results), \
               os.path.join(os.environ["HOME_DIR"], "results", f"{name_of_experiment}_{args['dataset']}.pt"))

# # thresholds = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
# # train_results = {x: [] for x in thresholds}
# # test_results = {x: [] for x in thresholds}
# # name_of_experiment = "fc1_thresholds"
    
# # for fc1_threshold in thresholds:
# #     for rs in range(70, 73):
# #         args["seed"] = rs
# #         args['sparsity'][2]['threshold'] = fc1_threshold
# #         print(f"----{rs}----{fc1_threshold}----")
# #         print(args['sparsity'])
# #         # args["save_name"] = f"{args['dataset']}_{rs}_{name_of_experiment}_{fc1_threshold}.pt"
# #         trained_model, device, train_loader, test_loader, criterion = main(args, train_args)
# #         train_acc, train_loss = test(trained_model, device, criterion, train_loader)
# #         test_acc, test_loss = test(trained_model, device, criterion, test_loader)
# #         train_results[fc1_threshold].append((train_acc, train_loss))
# #         test_results[fc1_threshold].append((test_acc, test_loss))
# #         torch.save((train_args, args, train_results, test_results), \
# #                    os.path.join(os.environ["HOME_DIR"], "results", f"{args['dataset']}-{name_of_experiment}.pt"))

----282----{'dataset': 'CIFAR10', 'init': 'signed_constant', 'batch_size': 128, 'epochs': 100, 'optimizer': 'SGD', 'optim_kwargs': {'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0001}, 'scheduler': True, 'no_cuda': False, 'seed': 282, 'save_name': 'FINAL_thresholds_0.02', 'sparsity': [{'threshold': 0.02, 'min_sparsity': 0.05}, {'threshold': 0.02, 'min_sparsity': 0.05}, {'threshold': 0.02, 'min_sparsity': 0.05}, {'threshold': 0.02, 'min_sparsity': 0.05}, {'threshold': 0.02, 'min_sparsity': 0.05}], 'copy_layers': [], 'bias': False}----
Using device cuda
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.299925
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.848503
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.801624
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.841756
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.834553
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.575366
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.816897
Train Epoch: 8 [0/50000 (0%)]	Loss: 1.661675
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.571058

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f81359f8e60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Test set: Average loss: 0.0017, Accuracy: 4066/10000 (41%)

Train Epoch: 81 [0/50000 (0%)]	Loss: 1.661972
Train Epoch: 82 [0/50000 (0%)]	Loss: 1.611493
Train Epoch: 83 [0/50000 (0%)]	Loss: 1.769854
Train Epoch: 84 [0/50000 (0%)]	Loss: 1.685572
Train Epoch: 85 [0/50000 (0%)]	Loss: 1.625749
Train Epoch: 86 [0/50000 (0%)]	Loss: 1.765705
Train Epoch: 87 [0/50000 (0%)]	Loss: 1.771898
Train Epoch: 88 [0/50000 (0%)]	Loss: 1.668178
Train Epoch: 89 [0/50000 (0%)]	Loss: 1.701783
Train Epoch: 90 [0/50000 (0%)]	Loss: 1.775818

Train set: Average loss: 0.0131, Accuracy: 20183/50000 (40%)


Test set: Average loss: 0.0017, Accuracy: 4085/10000 (41%)

Train Epoch: 91 [0/50000 (0%)]	Loss: 1.746077
Train Epoch: 92 [0/50000 (0%)]	Loss: 1.726413
Train Epoch: 93 [0/50000 (0%)]	Loss: 1.763391
Train Epoch: 94 [0/50000 (0%)]	Loss: 1.599362
Train Epoch: 95 [0/50000 (0%)]	Loss: 1.682956
Train Epoch: 96 [0/50000 (0%)]	Loss: 1.791580
Train Epoch: 97 [0/50000 (0%)]	Loss: 1.800119
Train Epoch: 98 [0/50000 (0%)]	Lo

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f81359f8e60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Test set: Average loss: 0.0018, Accuracy: 3790/10000 (38%)

Train Epoch: 21 [0/50000 (0%)]	Loss: 1.705016
Train Epoch: 22 [0/50000 (0%)]	Loss: 1.878831
Train Epoch: 23 [0/50000 (0%)]	Loss: 1.629217
Train Epoch: 24 [0/50000 (0%)]	Loss: 1.838748
Train Epoch: 25 [0/50000 (0%)]	Loss: 1.828609
Train Epoch: 26 [0/50000 (0%)]	Loss: 1.653798
Train Epoch: 27 [0/50000 (0%)]	Loss: 1.867588
Train Epoch: 28 [0/50000 (0%)]	Loss: 1.662352
Train Epoch: 29 [0/50000 (0%)]	Loss: 1.819248
Train Epoch: 30 [0/50000 (0%)]	Loss: 1.809580

Train set: Average loss: 0.0136, Accuracy: 19006/50000 (38%)



...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f81359f8e60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Test set: Average loss: 0.0015, Accuracy: 4633/10000 (46%)

Train Epoch: 31 [0/50000 (0%)]	Loss: 1.587117
Train Epoch: 32 [0/50000 (0%)]	Loss: 1.583768
Train Epoch: 33 [0/50000 (0%)]	Loss: 1.626969
Train Epoch: 34 [0/50000 (0%)]	Loss: 1.537219
Train Epoch: 35 [0/50000 (0%)]	Loss: 1.513204
Train Epoch: 36 [0/50000 (0%)]	Loss: 1.553804
Train Epoch: 37 [0/50000 (0%)]	Loss: 1.558520
Train Epoch: 38 [0/50000 (0%)]	Loss: 1.659153
Train Epoch: 39 [0/50000 (0%)]	Loss: 1.546718
Train Epoch: 40 [0/50000 (0%)]	Loss: 1.440726

Train set: Average loss: 0.0121, Accuracy: 23251/50000 (47%)



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f81359f8e60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Test set: Average loss: 0.0015, Accuracy: 4631/10000 (46%)

Train Epoch: 41 [0/50000 (0%)]	Loss: 1.621558
Train Epoch: 42 [0/50000 (0%)]	Loss: 1.547393
Train Epoch: 43 [0/50000 (0%)]	Loss: 1.583866


RuntimeError: ignored

In [7]:
import pickle
def save_study(study, filename):
    with open(os.path.join(os.environ["HOME_DIR"], "results", "studies", f"{filename}.pickle"), "wb") as f:
        pickle.dump(study, f)

In [15]:
def objective(trial):
    generated_seed = torch.randint(100, (1,)).item()
    trial.set_user_attr('seed', generated_seed)

    train_args = {
      "test_batch_size": 1000, # input batch size for testing (default: 1000)
      'data': '../data', # Location to store data (e.g. MNIST)
      'log_interval': 1000000, # how many batches to wait before logging training status
      'train_eval_interval': 20, # epoch interval at which to print training accuracy
      'test_eval_interval': 20, # epoch interval at which to print test accuracy
      'eval_on_last': True
    }

    args = {
      "dataset": "CIFAR10",
      "init": "signed_constant",
      "batch_size": 64, # input batch size for training (default: 64)
      "epochs": [trial.suggest_int(f'layer{i}_epochs', 20, 160) for i in range(5)], # number of epochs to train (default: 14)
      "optimizer": "SGD",
      "optim_kwargs": {"lr": trial.suggest_float('learning_rate', 0.01, 0.5), 
                       "momentum": trial.suggest_float('momentum', 0.2, 0.95),
                       "weight_decay": trial.suggest_float('weight_decay', 0.00005, 0.001)},
      "scheduler": True, # False for Adam, True for SGD, does CosineAnnealing
      'no_cuda': False, # disables CUDA training
      'seed': generated_seed, # random seed (default: 1)
      'save_name': None, # "simple20_rs2", # For Saving the current Model, None if not saving
      'sparsity': [
                   {"sparsity": trial.suggest_float('sparsity_conv1', 0.1, 0.95)}, 
                   {"sparsity": trial.suggest_float('sparsity_conv2', 0.1, 0.95)}, 
                   {"sparsity": trial.suggest_float('sparsity_fc1', 0.1, 0.95)}, 
                   {"sparsity": trial.suggest_float('sparsity_fc2', 0.1, 0.95)},
                   {"sparsity": trial.suggest_float('sparsity_fc3', 0.1, 0.95)}
                  ], # 'how sparse is each layer'
      'copy_layers': [], # ['conv1', 'conv2', 'fc2'],
      'bias': False
    }

    print(args)

    trained_model, device, train_loader, test_loader, criterion = main(args, train_args, trial=trial)
    train_acc, train_loss = test(trained_model, device, criterion, train_loader)
    test_acc, test_loss = test(trained_model, device, criterion, test_loader)

    return test_acc

In [ ]:
# with open(os.path.join(os.environ["HOME_DIR"], "results", "studies", "hp_search_study.pickle"), "rb") as f:
#     study = pickle.load(f)
study = optuna.create_study(direction='maximize', 
                            pruner=optuna.pruners.PatientPruner(
                                optuna.pruners.MedianPruner(n_startup_trials=15, n_warmup_steps=50, interval_steps=1), 
                                patience=40
                                )
                            )
for _ in range(100):
    study.optimize(objective, n_trials=1, show_progress_bar=True)
    save_study(study, "result__conv2_sparsity_per_layer_freezing")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  
[I 2022-05-03 05:30:26,633] A new study created in memory with name: no-name-3aab3e0c-e1ac-40c2-ab8d-80ad9f460c48
/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

{'dataset': 'CIFAR10', 'init': 'signed_constant', 'batch_size': 64, 'epochs': [62, 156, 42, 143, 119], 'optimizer': 'SGD', 'optim_kwargs': {'lr': 0.348635484985551, 'momentum': 0.7454563459246135, 'weight_decay': 0.0009874760015541415}, 'scheduler': True, 'no_cuda': False, 'seed': 97, 'save_name': None, 'sparsity': [{'sparsity': 0.10893508969416346}, {'sparsity': 0.38954459790126017}, {'sparsity': 0.7752736002325623}, {'sparsity': 0.4099074970417489}, {'sparsity': 0.857791243083931}], 'copy_layers': [], 'bias': False}
Using device cuda
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.297887
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.913256
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.790901
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.746760
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.905887
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.914496
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.843603
Train Epoch: 8 [0/50000 (0%)]	Loss: 1.876664
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.742581
Train Epoch: 10

/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'dataset': 'CIFAR10', 'init': 'signed_constant', 'batch_size': 64, 'epochs': [48, 24, 115, 134, 126], 'optimizer': 'SGD', 'optim_kwargs': {'lr': 0.4048351589221216, 'momentum': 0.47885585634203987, 'weight_decay': 0.000432813297043943}, 'scheduler': True, 'no_cuda': False, 'seed': 87, 'save_name': None, 'sparsity': [{'sparsity': 0.7883126900036797}, {'sparsity': 0.38271725131914586}, {'sparsity': 0.3044165190354116}, {'sparsity': 0.517700743708566}, {'sparsity': 0.38063061496931944}], 'copy_layers': [], 'bias': False}
Using device cuda
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.297189
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.590571
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.700707
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.494675
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.615847
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.696278
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.385802
Train Epoch: 8 [0/50000 (0%)]	Loss: 1.615256
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.289933
Train Epoch: 1

/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

{'dataset': 'CIFAR10', 'init': 'signed_constant', 'batch_size': 64, 'epochs': [41, 60, 100, 100, 139], 'optimizer': 'SGD', 'optim_kwargs': {'lr': 0.499647933387417, 'momentum': 0.4017029505495383, 'weight_decay': 0.000678328918814147}, 'scheduler': True, 'no_cuda': False, 'seed': 1, 'save_name': None, 'sparsity': [{'sparsity': 0.5388472467732387}, {'sparsity': 0.38877342036674284}, {'sparsity': 0.26204196923455186}, {'sparsity': 0.43243638353287356}, {'sparsity': 0.48329048360940663}], 'copy_layers': [], 'bias': False}
Using device cuda
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.307403
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.615005
Train Epoch: 3 [0/50000 (0%)]	Loss: 2.085586
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.534570
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.556433
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.254485
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.386694
Train Epoch: 8 [0/50000 (0%)]	Loss: 1.986539
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.298660
Train Epoch: 1